The EM algorithm formalizes a fairly old approach to handling missing data: first replace missing data by estimated values, then estimates the model parameters, and perhaps, repeat these two steps several times.

### Example 1. Genetic Linkage Model (Rao, 1973)
##### Problem
Suppose the 197 animals ($Y$) are distributed into four categories as follows: 
$$Y = (y_1, y_2, y_3, y_4) = (125, 18, 20, 34)$$
with cell probabilities .   
$$\big( \frac{1}{2}+\frac{\theta}{4}, \frac{1}{4}(1-\theta), \frac{1}{4}(1-\theta), \frac{\theta}{4} \big)$$
The observed data likelihood: 
$$g(Y\ |\ \theta) \propto \big(\frac{1}{2}+\frac{\theta}{4}\big)^{y_1} \big(\frac{1}{4}(1-\theta)\big)^{y_2+y_3} \big(\frac{\theta}{4}\big)^{y_4}$$

Augment the observed data by splitting the first cell into two cells with probabilities $\frac{1}{2}$ and $\frac{\theta}{4}$. The augmented data are given by 
$$(x_1,x_2,x_3,x_4,x_5)$$
such that 
$$
\begin{aligned}
x_1+x_2 &= y_1=125  \\
    x_3 &= y_2=18  \\
    x_4 &= y_3=20  \\
    x_5 &= y_4=34
\end{aligned}
$$

The "missing data" $Z(=x_2)$ is like a coin toss for which category each observation of $y_1$ gets split into:  
$$Z \sim Binomial(125, p), \text{where } p=\frac{\theta/4}{1/2 + \theta/4} = \frac{\theta}{2+\theta}$$
The augmented data likelihood is now simpler: 
$$f(X, Z\ |\ \theta) \propto (1-\theta)^{x_3+x_4} \theta^{x_2+x_5}$$
**E-step. **
Now we compute the Q function (under a flat prior):  
$$
\begin{aligned}
Q(\theta, \theta^{old}) = E_{Z | X, \theta^{old}} \log f(X\ |\ \theta) &= \log \theta \big(E_{Z | Y, \theta^{old}}[x_2] + x_5 \big) + (x_3+x_4)\log (1-\theta)  \\
&= \log \theta \big(125 \times \frac{\theta^{old}}{2+\theta^{old}} + 34 \big) + (18+20)\log (1-\theta)
\end{aligned}
$$
where the second equality follows from the mean of a binomial distribution.   
**M-step. **
$$
\begin{aligned}
\frac{\partial Q(\theta, \theta^{old})}{\partial \theta} \bigg|_{\theta=\theta^{old}} = 0 &\Rightarrow \big(125 \times \frac{\theta^{old}}{2+\theta^{old}} + 34 \big) \times \frac{1}{\theta} - \frac{38}{1-\theta}=0  \\
&\Rightarrow \theta = \frac{125 \times \frac{\theta^{old}}{2+\theta^{old}} + 34}{125 \times \frac{\theta^{old}}{2+\theta^{old}} + 34 + 38}  \text{ (recursive formula) }
\end{aligned}
$$

##### Discussion
Starting at $\theta^0 = 0.5$, the EM algorithm converges to $\theta^*=0.6268$ (the observed posterior mode) after a few iterations. One might ask can we take the derivate of the observed data likelihood (original data likelihood) and set it to zero to get the answer? The answer is yes. The derivate is given as follow:  
$$\frac{d}{d \theta} \log g(\theta | Y) = \frac{125}{2+\theta} - \frac{38}{1-\theta} + \frac{34}{\theta} = 0$$
It is easy to see that $\theta^*$ satisfies this equation (see M-step). Which means, in this case, EM algorithm yields the same result as the MLE.  

Another question is how to estimate the variance of $\theta^*$? Statisticians are not satisfied with only the estimaed value of $\mu$, they also want to know the variance of the estimate. In general, there are two kind of methods: (1) Parametric Bootstrap (2) Louis' Method [2]  
(1) Parametric Bootstrap:  
- Generate random Y based on $\mu^*$   
- Run EM algorithm on each bootstrap sample   
- Calculate the bootstrap variance of the estimated $\mu$    

(2) Louis' Method [2] (TODO)   

In [1]:
import numpy as np
np.random.seed(20170320)
# generate parametric bootstrap sample based on theta_star
theta_star = 0.6268
probs = [0.5+theta_star/4, 0.25*(1-theta_star), 0.25*(1-theta_star), theta_star/4]
bootstrap_sample = np.random.multinomial(197, probs, size=100)

# run EM-algorithm on each bootstrap sample
def EM(y, t=0.5, rounds=10):
    for _ in range(rounds):
        t = (y[0]*(t)/(2+t)+y[3]) / (y[0]*(t)/(2+t)+y[3]+y[1]+y[2])
    return t
# standard deviation of EM-estimates of bootstrap sample
np.std(np.apply_along_axis(EM, 1, bootstrap_sample))

0.05042808756634462

### Example 2. Linear Regression with Right-Censored Data
We consider the motorette data found in Schmee and Hahn (1979). Ten motorettes were tested at each of the four temperatures: 150C, 170C, 190C and 220C. The time to failure in hours is given in Table below.   

<img src="motorette.png" alt="motorette_data" style="width: 60%;"/>

A star indicates that a motorette was taken off the study without failing at the event time indicated. For these data, we will fit the model:   
$$t_i = \beta_0 + \beta_1 \nu_i + \sigma \epsilon_i$$   
where $\epsilon_i \sim N(0,1)$, $\nu_i=1000/(\text{temperature} + 273.2)$ and $t_i=\log_{10}$ ($i$th failure time).   
Reorder the data so that the first $k$ observations are uncensored (i.e. a failure is observed at $t_i$) and the remaining $n-k$ are censored ($c_i$ denotes a censored event time). The log-augmented posterior (under a flat prior) is given (up to a constant by):    
$$-n\log\sigma - \sum_{i=1}^k(t_i-\beta_0-\beta_1\nu_i)^2/2\sigma^2 - \sum_{i=k+1}^{n}(Z_i-\beta_0-\beta_1\nu_i)^2/2\sigma^2$$   
where $Z_j$ is the (unobserved) failure time for case $j$. The conditional predictive distribution $p(Z_i\ |\ \beta_0, \beta_1, \sigma, c_i)$ is the conditional normal distribution, conditional on the fact that the unobserved failure time $Z_j$ is greater than $c_j$.   
**E-step .** Compute:  
$$
\begin{aligned}
-n\log\sigma &- \frac{1}{2\sigma^2}\sum_{i=1}^{k}(t_i-\beta_0-\beta_1\nu_i)^2 \\
&- \frac{1}{2\sigma^2}\sum_{i=k+1}^{n}\bigg[  E[Z_i^2\ |\ \beta_0^{old}, \beta_1^{old}, \sigma^{old}, Z_i > c_i] - 2(\beta_0+\beta_1)E[Z_i\ |\ \beta_0^{old}, \beta_1^{old}, \sigma^{old}, Z_i > c_i]+(\beta_0+\beta_1\nu_i)^2 \bigg]
\end{aligned}
$$

Note that: 
$$ E[Z_i^2\ |\ \beta_0^{old}, \beta_1^{old}, \sigma^{old}, Z_i > c_i] = \mu_i^2 + \sigma^2 + \sigma (c_i+\mu_i) H\bigg(\frac{c_i-\mu_i}{\sigma} \bigg)$$
and   
$$ E[Z_i\ |\ \beta_0^{old}, \beta_1^{old}, \sigma^{old}, Z_i > c_i] = \mu_i + \sigma H\bigg(\frac{c_i-\mu_i}{\sigma} \bigg) $$  
where $\mu_i = \beta_0+\beta_1\nu_i$, $H(x)=\phi(x)/(1-\Phi(x))$, and $\phi(x)$ and $\Phi(x)$ are the density and cdf of the standard normal distribution.    
**M-step .**  
$$
\begin{aligned}
\frac{\partial Q}{\partial\beta_0} &= 0 \rightarrow \sum_{i=1}{m}(t_i-\beta_0-\beta_1\nu_i) + \sum_{i=k+1}^{n}\big[ E(Z_i)-\beta_0-\beta_1\nu_i \big] = 0  \\
\frac{\partial Q}{\partial\beta_1} &= 0 \rightarrow \sum_{i=1}{m}\nu_i(t_i-\beta_0-\beta_1\nu_i) + \sum_{i=k+1}^{n}\nu_i\big[ E(Z_i)-\beta_0-\beta_1\nu_i \big] = 0  \\
\frac{\partial Q}{\partial\sigma^2} &= 0 \rightarrow \frac{\sum_{i=1}^k(t_i-\beta_0-\beta_1\nu_i)^2}{\sigma^4} + \frac{\sum_{i=k+1}^n \big[ E(Z_i)^2-2(\beta_0+\beta_1\nu_i)E(Z_i)+(\beta_0+\beta_1\nu_i)^2 \big]}{\sigma^4} - \frac{n}{\sigma^4} = 0
\end{aligned}
$$

To obtain $\beta^{new}$, replace $c_j$ by $E[Z_j\ |\ \beta_0^{old}, \beta_1^{old},\sigma^{old}, Z_j>c_j]$ and apply least squares. To obtain $\sigma_{new}^2$, compute:  
$$\sigma_{new}^2 = \frac{\sum_{j=1}^k(t_j-\mu_j^{old})^2}{n} + \sigma_{old}^2 \frac{\sum_{j=k+1}^n \big[ 1+\big(\frac{c_j-\mu_j^{old}}{\sigma_{old}}H\big( \frac{c_j-\mu_j^{old}}{\sigma_{old}} \big) \big]}{n}$$  
where $\mu_j^{old} = \beta_0^{old}+\beta_1^{old}\nu_j$

In [2]:
import pandas as pd
motorette = pd.read_csv("motorette.csv")
motorette.transpose()

,0,1,2,3,4,5,6,7,8,9
150C,8064*,8064*,8064*,8064*,8064*,8064*,8064*,8064*,8064*,8064*
170C,1764,2772,3444,3542,3780,4860,5196,5448*,5448*,5448*
190C,408,408,1344,1344,1440,1680*,1680*,1680*,1680*,1680*
220C,408,408,504,504,504,528*,528*,528*,528*,528*


### Example 3. Normal distribution with unknown mean and variance and partially conjugate prior distribution
##### Introduction
This example comes from Gelman's book[1 Page 322], which shows the application of EM algorithm in Bayesian theory. Bayesian computation revolves around two steps: computation of the posterior distribution, $p(\theta\ |\ y)$, and computation of the posterior predictive distribution, $p(\tilde{y}\ |\ y)$. In problems with many parameters, it is very often we are only interested in a $subset$ of the parameters; we use the notation $\theta = (\gamma, \phi)$ and suppose we are interested in approximating $p(\phi \ |\ y)$. Then the EM algorithm can be viewed as an iterative method for finding the mode of the marginal posterior density, $p(\phi \ |\ y)$.   
##### Problem
Suppose we weigh an object on a scale $n$ times, and the weighings, $y_1, \dots, y_n$, are assumed independent with a $N(\mu, \sigma^2)$ distribution, where $\mu$ is the true weight of the object. And we assume a $N(\mu_0, \tau_0^2)$ prior distribution on $\mu$ (with $\mu_0$ and $\tau_0$ known) and a non-informative uniform prior distribution on $\log \sigma$ (that is $p(\sigma) = 1/\sigma$); these form a partially conjugate joint prior distribution.    
##### Discussion
Because the model is not fully conjugate, there is no standard form for the joint posterior distribution of $(\mu, \sigma)$ and no closed-form expression for the marginal posterior density $\mu$. In this problem, since there are only two parameters we can easily do a grid search over $\mu$ and $\sigma$ and then create a simple contour plot. But if we are only interested in $\mu$, we can, however, think of $\mu$ as the parameter in our problem and $\sigma$ as missing data. EM can therefore be used to find the marginal posterior mode of $\mu$, averaging over $\sigma$; that is, $(\mu, \sigma)$ corresponds to $(\phi, \gamma)$ in the general notation.    
##### Solution
Model:  
$$
\begin{aligned}
& y\ |\ \mu, \sigma^2 \sim N(\mu, \sigma^2)   \\
& \mu\ |\ \mu_0, \tau_0^2 \sim N(\mu_0, \tau_0^2)   \\
& \log \sigma \propto 1
\end{aligned}
$$

Write down the joint distribution: 
$$
\begin{aligned}
p(y, \mu, \sigma) &= p(y\ |\ \mu, \sigma) \times p(\mu\ |\ \mu_0, \tau_0) \times p(\sigma)   \\
                  &= \prod_{i=1}^n  \frac{1}{\sqrt{2\pi} \sigma} exp\big({-\frac{1}{2\sigma^2}(y_i-\mu)^2}\big) \times \frac{1}{\sqrt{2\pi} \tau_0} exp\big({-\frac{1}{2\tau_0^2}(\mu-\mu_0)^2}\big) \times \frac{1}{\sigma}
\end{aligned}
$$

Posterior distribution (ignoring terms that do not depend on $\mu$ or $\sigma^2$):  
$$p(\mu, \sigma \ |\ y) \propto \big(\frac{1}{\sigma}\big)^{(n+1)} \prod_{i=1}^n exp\big({-\frac{1}{2\sigma^2}(y_i-\mu)^2}\big) \times exp\big({-\frac{1}{2\tau_0^2}(\mu-\mu_0)^2}\big)$$

Joint log posterior density:  
$$\log p(\mu, \sigma \ |\ y) = -\frac{1}{2\tau_0^2}(\mu-\mu_0)^2 - (n+1)\log \sigma - \frac{1}{2\sigma^2} \sum_{i=1}^n (y_i-\mu)^2 + constant$$

**E-step.** For the E-step of the EM algorithm, we must determine the expectation of the joint log posterior density, averaging over $\sigma$ and conditional on the current guess $\mu^{old}$ and y:  

$$E_{\sigma\ |\ \mu_{old},y} \log p(\mu, \sigma\ |\ y) = -\frac{1}{2\tau_0^2}(\mu-\mu_0)^2 - (n+1) E_{\sigma\ |\ \mu_{old},y} [\log \sigma] - \frac{1}{2}E_{\sigma\ |\ \mu_{old},y} \big[ \frac{1}{\sigma^2} \big] \sum_{i=1}^n (y_i-\mu)^2 + constant$$   

Actually, we need evaluate only the second expectation $E_{\sigma\ |\ \mu_{old},y} [ \frac{1}{\sigma^2} ]$, because the first expectation does not involve $\mu$ and thus will not affect the M-step. The expectation can be evaluated by noting that, given $\mu$, the posterior distribution of $\sigma^2$ is scaled inverse-chi-squared:  

$$\sigma^2\ | \ \mu, y \sim Inv-\chi^2\big(n, \frac{1}{n}\sum_{i=1}^n(y_i-\mu)^2\big)$$

Then the conditional posterior distribution of $\frac{1}{\sigma^2}$ is a scaled $\chi^2$, and  
$$E_{\sigma\ |\ \mu_{old},y} \big[ \frac{1}{\sigma^2} \ \big|\ \mu_{old}, y \big] = \bigg( \frac{1}{n} \sum_{i=1}^n(y_i-\mu^{old})^2 \bigg)^{-1}$$
We can then re-express the expectation as:  
$$E_{\sigma\ |\ \mu_{old},y} \log p(\mu, \sigma\ |\ y) = -\frac{1}{2\tau_0^2}(\mu-\mu_0)^2 - \frac{1}{2}\bigg( \frac{1}{n} \sum_{i=1}^n(y_i-\mu^{old})^2 \bigg)^{-1} \sum_{i=1}^n (y_i-\mu)^2 + constant$$  
**M-step.**  For the M-step, we must find the $\mu$ that maximizes the above expression. The task is straightforward, simply take the derivate and set it to 0, then we have:  
$$\mu^{new} = \frac{\frac{1}{\tau_0^2}\mu_0 + \frac{n}{\frac{1}{n}\sum_{i=1}^n(y_i-\mu^{old})^2}\bar{y} }{\frac{1}{\tau_0^2} + \frac{n}{\frac{1}{n}\sum_{i=1}^n(y_i-\mu^{old})^2} }$$   
If we iterate this computation, $\mu$ converges to the marginal mode of $p(\mu\ |\ y)$.

References:    
[1] Gelman, Andrew, et al. Bayesian data analysis. Vol. 2. Boca Raton, FL, USA: Chapman & Hall/CRC, 2014.    
[2] Louis, Thomas A. "Finding the observed information matrix when using the EM algorithm." Journal of the Royal Statistical Society. Series B (Methodological) (1982): 226-233.    


In [3]:
import numpy as np
np.random.seed(20170320)

mu_0 = 0
tau_0 = 1
y = np.random.normal(2, 5, size=100)
def log_post_mu_sigma(mu, sigma, y): 
    return -1.0/(2*tau_0**2) * (mu-mu_0)**2 - (len(y)+1)*np.log(sigma) - 1.0/(2*sigma**2) * sum((y-mu)**2)

grid_mu = np.arange(-5, 5, 0.05)
grid_sigma = np.arange(0.1, 10, 0.05)
z = [[log_post_mu_sigma(m, s, y) for m in grid_mu] for s in grid_sigma]
z = np.exp(z) / np.sum(np.exp(z))

In [4]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

data = [go.Contour(z=z, 
                   x=grid_mu, 
                   y=grid_sigma, 
                   colorscale=[[0, 'rgb(254,251,251)'], [0.5, 'rgb(253, 197, 142)'], [1,'rgb(227,54,31)']])]

layout = go.Layout(title='Posterior Distribution of mu and sigma', 
                   xaxis=dict(range=[0,3], title='Mu'), 
                   yaxis=dict(range=[4,6.5], title='Sigma'), 
                   showlegend=False)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [5]:
def mu_update(mu_old):
    numerator = 1/tau_0**2 * mu_0 + np.sum(y) / np.mean((y-mu_old)**2)
    denominator = 1/tau_0**2 + len(y) / np.mean((y-mu_old)**2)
    return numerator/denominator

m = 0
for _ in range(10):
    m = mu_update(m)
print m

1.36996896969


In [6]:
from IPython.core.display import HTML
HTML("<style>div.text_cell_render{font-family:'Times New Roman';font-size:1.3em;line-height:1.9em;}</style>")